In [44]:
import pandas as pd

from energymercato.entities.power import *
from energymercato.entities.player import *
from energymercato.usecases.simulation import *
import random
from energymercato.usecases.usecases import (
    make_plant, 
    make_storage_plant, 
    penalty_amount, 
    create_table,
    create_df_power,
    sales_h,
    compute_balance
    )

In [45]:
# Parameter definition

path = "/mnt/g/Mon Drive/SEM/energymercato"
from pathlib import Path
print("path ",str(Path(path).parent), "is existing ?",Path(path).parent.exists())


solo = False

if solo:
    random.seed(10)

# Define client mwh_price
client_price = 50
penalty_price_j1 = 200
arenh_price = 42
arenh_proportion = .2

# Define players
p1 = 900
p2 = 1300
p3 = 1450

path  /mnt/g/Mon Drive/SEM is existing ? True


In [46]:
# Define power plant (inspired by Wikipedia power plant informations in France)
print("creation of plant factory...")
make_nuke_plant = make_plant(NukePlant, .4,.95,35, 3000)
make_coal_plant = make_plant(CoalPlant, .2,.9, 45, 500)
make_gas_plant = make_plant(GasPlant, .2,.9,55, 1000)
make_wind_plant = make_plant(WindPlant, 0,1,0, 0)
make_solar_plant = make_plant(SolarPlant, 0,1,0, 0)
make_water_plant = make_storage_plant(StoragePlant, 1,1,0,0,3000,20,0)
make_hydro_plant = make_plant(HydroPlant, 0,1,10, 0)

print("creation of specific plant...")
grave = make_nuke_plant(6*p1, "grave")
choo = make_nuke_plant(1*p3, "choo")
civ = make_nuke_plant(2*p3, "civ")
catt = make_nuke_plant(4*p2, "catt")
palu = make_nuke_plant(4*p2, "palu")
pen = make_nuke_plant(2*p2, "pen")
tri = make_nuke_plant(4*p1, "tri")
bug = make_nuke_plant(4*p1, "bug")
cru = make_nuke_plant(4*p1, "cru")
bla = make_nuke_plant(4*p1, "bla")
chin = make_nuke_plant(4*p1, "chin")
corde = make_coal_plant(1200, "corde")
hydro_ndf = make_hydro_plant(2000, "hydro_ndf")
fossette = make_solar_plant(12, "fossette")
belfays = make_wind_plant(20, "belfays")
marne = make_wind_plant(220, "marne")

prov = make_coal_plant(1200, "prov")
huchet = make_coal_plant(600, "huchet")

fos = make_gas_plant(490, "fos")
golfe = make_gas_plant(435, "golfe")
mont = make_gas_plant(435, "mont")
choo2 = make_nuke_plant(1*p3, "choo2")
cvent = make_wind_plant(435, "cvent")
cvent_sol = make_solar_plant(88, "cvent_sol")
solaire_direct = make_solar_plant(1500, "solaire_direct")
hydro_ngie = make_hydro_plant(1800, "hydro_ngie")

cnr_wind = make_wind_plant(800, "cnr_wind")
cnr_solar = make_solar_plant(200, "cnr_solar")
cnr_water = make_hydro_plant(3000, "cnr_water")

huch6 = make_coal_plant(595, "huch6")
prov5 = make_coal_plant(595, "prov5")
gazel_wind = make_wind_plant(83.5, "gazel_wind")
gazel_solar = make_solar_plant(10.5, "gazel_solar")

t_wind = make_wind_plant(600, "t_wind")
t_sol = make_solar_plant(480, "t_sol")
t_hydro = make_hydro_plant(150, "t_hydro")
marcinelle = make_gas_plant(500, "marcinelle")
bayet = make_gas_plant(400, "bayet")

sw_hydro = make_hydro_plant(400, "sw_hydro")

creation of plant factory...
creation of specific plant...


In [48]:
# define players

print("defining players...")
if solo : 
    print("solo mode !")
    ndf = Player(name="ndf",score = [0],plant_list=[
        grave,choo,civ,hydro_ndf
    ], client_proportion = .6, 
    client_price_mwh=client_price,path=path)
    ngie = Player(name="ngie",score = [0],plant_list=[
        fos, mont, golfe, hydro_ngie
    ], client_proportion = .15,client_price_mwh=client_price, path=path)
    gazel = Player(name="gazel",score = [0],plant_list=[
            gazel_wind, gazel_solar
        ], client_price_mwh=client_price, path=path)
else :
    ndf = Player(name="ndf",score = [0],plant_list=[
        grave,choo,civ, catt, palu, pen,
        tri, bug,cru,bla,chin, corde,
        hydro_ndf,fossette,belfays,marne
    ], client_proportion = .5,client_price_mwh=client_price, path=path)
    ngie = Player(name="ngie",score = [0],plant_list=[
        fos, mont, golfe,choo2, cvent, 
        cvent_sol, solaire_direct, hydro_ngie
    ], client_proportion = .15,client_price_mwh=client_price, path=path)
    gazel = Player(name="gazel",score = [0],plant_list=[
        huch6,prov5, gazel_wind, gazel_solar
    ], client_proportion = .05, client_price_mwh=client_price,path=path)


cnr = Player(name = "cnr", score = [0],plant_list=[
    cnr_wind,cnr_solar,cnr_water
], client_proportion=0.1, client_price_mwh=client_price, path=path)


totalnrj = Player(name = "totalnrj",score = [0],plant_list=[
    t_sol,t_wind,t_hydro, marcinelle, bayet  
], client_proportion = .15, client_price_mwh=client_price, path=path)


alpiq = Player(name="alpiq", plant_list=[sw_hydro],client_proportion = .05, client_price_mwh=client_price, path=path)


defining players...


In [49]:
# Define playes for game and print total power
players = [
    ndf, ngie, gazel, cnr, alpiq, totalnrj, 
]

print("players selected : ", [str(f) for f in players])

total_power = sum(c.get_prod() for c in players)
total_power_enr = sum(c.get_prod(enr_only=True) for c in players)

print("total_power: ", total_power)
print("total_power_enr: ", total_power_enr," ", total_power_enr/total_power*100,"%")

print("writing power plant summary for players")
for p in players:
    p.write_power_plant()

players selected :  ['ndf', 'ngie', 'gazel', 'cnr', 'alpiq', 'totalnrj']
total_power:  56074.0
total_power_enr:  4449.0   7.934158433498591 %
writing power plant summary for players


In [50]:
# set date
from datetime import datetime
dt = datetime(2022,2,28)

# Define simulation
simu_w = SimulationWeather()
simu_p = SimulationPowerConsumption(sum_p_max=(total_power-total_power_enr)*.8)


In [11]:
(total_power-total_power_enr)*.8

18150.825

In [51]:
# Beginning of game round here

# dt increment
dt += timedelta(days=1)
# simulation

simu_w.generate_j(dt)

simu_p.generate_j(dt.day, dt.month, simu_w.Tmean)

simu_p.player_simu_repartition(
    proportion_list=[p.client_proportion for p in players],
    names = [p.name for p in players],
)

simu_p.j_curve.to_csv(path+"/conso_"+dt.isoformat()+".csv", sep=";",index=False)

simu_w.get_df_j_curve().to_csv(path+"/meteo_"+dt.isoformat()+".csv", sep=";",index=False)

# step get enr curves
for p in players:
    p.get_p(simu_w.j_curve)

# creation table
df_power_0 = create_table(players)
df_power = df_power_0.copy()

In [33]:
# df_power0 = create_table(players)

In [13]:
players[0].j_prev

,Name,type,p_min,p_max,mwh_cost,hour,p,player
0,grave,NukePlant,2160.0,5130.0,60,0,0,ndf
1,choo,NukePlant,580.0,1377.5,60,0,0,ndf
2,civ,NukePlant,1160.0,2755.0,60,0,0,ndf
3,hydro_ndf,HydroPlant,0,2054,30,0,0,ndf
4,grave,NukePlant,2160.0,5130.0,60,1,0,ndf
...,...,...,...,...,...,...,...,...
91,hydro_ndf,HydroPlant,0,2159,30,22,0,ndf
92,grave,NukePlant,2160.0,5130.0,60,23,0,ndf
93,choo,NukePlant,580.0,1377.5,60,23,0,ndf
94,civ,NukePlant,1160.0,2755.0,60,23,0,ndf


In [35]:
# df_power0[df_power0.hour ==0]

,Name,type,p_min,p_max,mwh_cost,hour,p,player
193,gazel_solar,SolarPlant,0.0,0,0,0,0,gazel
241,cnr_solar,SolarPlant,0,0,0,0,0,cnr
192,gazel_wind,WindPlant,0.0,240,0,0,0,gazel
240,cnr_wind,WindPlant,0,2252,0,0,0,cnr
99,hydro_ngie,HydroPlant,0,1955,30,0,0,ngie
3,hydro_ndf,HydroPlant,0,2054,30,0,0,ndf
242,cnr_water,HydroPlant,0,3271,30,0,0,cnr
1,choo,NukePlant,580.0,1377.5,60,0,0,ndf
2,civ,NukePlant,1160.0,2755.0,60,0,0,ndf
0,grave,NukePlant,2160.0,5130.0,60,0,0,ndf


In [15]:
simu_p.j_curve.Consommation

0     21844
1     21057
2     20891
3     20521
4     20202
5     20290
6     20967
7     21925
8     22785
9     23124
10    23282
11    23201
12    23272
13    23149
14    22696
15    22248
16    22027
17    22149
18    23150
19    23708
20    23349
21    22659
22    22111
23    22693
Name: Consommation, dtype: int64

In [52]:

# fixing 
sum_conso = simu_p.j_curve.Consommation
j_prev_fixing = {}
j_prev_power_fixed = {}

# df_power = create_df_power(players)
for j, i in enumerate(sum_conso):
    df_power_j = create_df_power(df_power, j)
    
    id_list = df_power_j[df_power_j.cumsum_p_max < i].index.tolist()
    if max(id_list) < len(df_power_j)-1:
        id_list.append(max(id_list)+1)
    j_prev_power_fixed[j] = df_power_j.iloc[id_list,:]

j_prev_fixing = [j.mwh_cost.max() for i,j in j_prev_power_fixed.items()]

d_fixing = pd.DataFrame(j_prev_fixing,columns=["mwh_price"])
d_fixing["hour"] = np.arange(len(d_fixing))
d_fixing["buy_mwh"] = 0
for p in players:
    d_fixing.to_csv(path+"/"+p.name+"/"+p.name+"_j_cmd.csv",sep=";", index=False)

/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.loc[:,"cumsum_p_max"] = d.loc[:,"p_max"].cumsum()
/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.loc[:,"cumsum_p_max"] = d.loc[:,"p_max"].cumsum()
/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:117: SettingWithCopyWarning: 
A value is trying to be set on

In [ ]:
print("Pause for filling")

In [20]:
# execute after some time for filling csv
for p in players:
    p.read_j_prev(path+"/"+p.name+"/"+p.name+"_j_prev.csv")

for p in players:
    p.read_j_market_cmd(path+"/"+p.name+"/"+p.name+"_j_cmd.csv", d_fixing)
    

/home/projects/energymercato/energymercato/src/energymercato/entities/player.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj.p[dj.p > dj.p_max] = dj.p_max[dj.p > dj.p_max]
/home/projects/energymercato/energymercato/src/energymercato/entities/player.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj.p[dj.p < dj.p_min] = dj.p_min[dj.p < dj.p_min]
/home/projects/energymercato/energymercato/src/energymercato/entities/player.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [21]:
dt

datetime.datetime(2022, 3, 1, 0, 0)

In [23]:
# scores :
df_power = compute_balance(players, df_power, dt, penalty_price_j1,simu_p)
sales_h(df_power, d_fixing, players, dt,arenh_price, arenh_proportion)

for p in players:
    p.print_score("score")

/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["amount"] = -a["mwh_cost"] * a["p"]
/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["amount"] = -a["mwh_cost"] * a["p"]
/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

PermissionError: [Errno 13] Permission denied: '/mnt/g/Mon Drive/game/ndf/ndf_score.csv'

#### After J-1 prediction, here is real time market :

In [24]:
from energymercato.usecases.usecases import read_spot, compute_score_hour

# Parameters
spot_price = 300
df_real = df_power.copy()
transaction_price = 300
h = -1 # hour
print("ready for next phase")

In [26]:
df_real

,Name,type,p_min,p_max,mwh_cost,hour,p,player
0,gazel_solar,SolarPlant,0.0,0,0,0,0.0,gazel
1,cnr_solar,SolarPlant,0,0,0,0,0.0,cnr
2,gazel_wind,WindPlant,0.0,240,0,0,0.0,gazel
3,cnr_wind,WindPlant,0,2252,0,0,2252.0,cnr
4,hydro_ngie,HydroPlant,0,1955,30,0,1955.0,ngie
...,...,...,...,...,...,...,...,...
307,civ,NukePlant,1160.0,2755.0,60,23,1525.5,ndf
308,grave,NukePlant,2160.0,5130.0,60,23,5130.0,ndf
309,mont,GasPlant,87.0,391.5,150,23,391.5,ngie
310,golfe,GasPlant,87.0,391.5,150,23,391.5,ngie


In [43]:
# Begining of 1/24 game round 
h +=1

dd = df_real[df_real.hour == h]
res = {}
balance = 2

sr=[]
for p in players:
    res = p.get_p_j(h, simu_w.j_curve[h])
    sr.append(res)
    
dd = dd.merge(pd.concat(sr,ignore_index=True), how="left", on=["player","Name","hour"])

dd = dd.fillna(0.0)

sum_power = dd.power.sum()
balance = abs(simu_p.j_curve.Consommation[h] - sum_power)

transaction = []

dd["spot"] = 0
dd.to_csv(path+"/"+"spot_"+str(h)+"h.csv", sep=";")



ValueError: cannot convert float NaN to integer

In [32]:
df_real[df_real.hour == h]

,Name,type,p_min,p_max,mwh_cost,hour,p,player
0,gazel_solar,SolarPlant,0.0,0,0,0,0.0,gazel
1,cnr_solar,SolarPlant,0,0,0,0,0.0,cnr
2,gazel_wind,WindPlant,0.0,240,0,0,0.0,gazel
3,cnr_wind,WindPlant,0,2252,0,0,2252.0,cnr
4,hydro_ngie,HydroPlant,0,1955,30,0,1955.0,ngie
5,hydro_ndf,HydroPlant,0,2054,30,0,2054.0,ndf
6,cnr_water,HydroPlant,0,3271,30,0,3271.0,cnr
7,choo,NukePlant,580.0,1377.5,60,0,1377.5,ndf
8,civ,NukePlant,1160.0,2755.0,60,0,1160.0,ndf
9,grave,NukePlant,2160.0,5130.0,60,0,5130.0,ndf


In [37]:
dd = read_spot(path+"/"+"spot_"+str(h)+"h.csv", dd)
dd["spot_changes"] = dd.power - dd.spot
transaction.append(dd.player[dd.spot_changes != 0].to_list())
sum_power = dd.spot.sum()
balance = abs(simu_p.j_curve.Consommation[h] - sum_power)
print(balance)

AttributeError: 'DataFrame' object has no attribute 'power'

In [28]:
balance

20611

In [38]:
# while balance > 1 :

compute_score_hour(players,transaction, transaction_price, h, spot_price, dd, dt)

print(f"h={h} go to next hour...",)

if h == 24:
    print("end of day : please go to next j-1")


AttributeError: 'DataFrame' object has no attribute 'spot_changes'